## Task3

In [ ]:
import pandas as pd
from UsefulFunctions import LoadData, PricesDK, Optimizer

### 3.1

In [ ]:
# Load the necessary data
df_prices, df_pro = LoadData()

# Compute Buy Prices using the given function
df_prices = PricesDK(df_prices)

# Keep only necessary columns
df_prices = df_prices[["HourDK", "Buy" , "Sell"]] # Keep only the Buy and Sell columns
df_pro = df_pro[["HourDK", "Load", "PV"]] # Consumption culumn is renamed to Load in the PricesDK function

# Merge both datasets on time
df = pd.merge(df_pro, df_prices, on="HourDK", how="inner")
del df_pro, df_prices # Remove now unused dataframes

# Convert time column to datetime format
df["HourDK"] = pd.to_datetime(df["HourDK"])

# Compute total cost for each year (Consumption * Buy Price per hour)
df["TotalCost"] = df["Load"] * df["Buy"]
annual_total_costs = df.groupby(df["HourDK"].dt.year)["TotalCost"].sum()

# Compute total yearly consumption and average Buy Price
annual_consumption = df.groupby(df["HourDK"].dt.year)["Load"].sum()
annual_avg_buy_price = df.groupby(df["HourDK"].dt.year)["Buy"].mean()

# Compute estimated cost using total consumption * average Buy Price
rough_costs = annual_consumption * annual_avg_buy_price

# Analyze the difference
difference = abs(annual_total_costs - rough_costs)

# Compare the two methods
comparison_df = pd.DataFrame({
    "Exact Total Cost": annual_total_costs,
    "Total Consumption (kWh)": annual_consumption,
    "Average Buy Price (DKK/kWh)": annual_avg_buy_price,
    "Estimated Total Cost (Rough Calculation)": rough_costs,
    "Difference": difference
})

# Print the final comparison table
print(comparison_df)

### 3.2 & 3.3

In [ ]:
# Load data
df_prices, df_pro = LoadData()
df_prices = PricesDK(df_prices)

# Battery parameters
battery_params = {
    'Pmax': 5,      # Power capacity in kW
    'Cmax': 10,     # Energy capacity in kWh
    'Cmin': 1,      # Minimum SOC (10%) 
    'C_0': 5,       # Initial SOC (50%)
    'C_n': 5,       # Final SOC (50%)
    'n_c': 0.95,    # Charging efficiency
    'n_d': 0.95     # Discharging efficiency
}

# Initialize results dictionaries
costs_nothing = {'2022': annual_total_costs[2022], '2023': annual_total_costs[2023]}
costs_pv_only = {'2022': 0, '2023': 0}
costs_battery_only = {'2022': 0, '2023': 0}

# Process data by days
days = pd.to_datetime(df_prices['HourDK'].dt.date.unique())
for day in days:
    year = day.year
    day_date = day.date()
    
    # Filter data for the current day
    prices_day = df_prices[df_prices['HourDK'].dt.date == day_date]
    pro_day = df_pro[df_pro['HourDK'].dt.date == day_date]
    
    # Make sure we have both price and prosumer data for this day
    if len(prices_day) == 0 or len(pro_day) == 0:
        continue
        
    # Make sure the timestamps match
    merged = pd.merge(prices_day, pro_day, on='HourDK')
    if len(merged) == 0:
        continue
    
    # Extract data
    buy_prices = merged['Buy'].values
    sell_prices = merged['Sell'].values
    load = merged['Load'].values
    generation = merged['PV'].values
    
    # Calculate net load (positive means consumption exceeds generation)
    net_load = load - generation
    
    # Calculate cost with PV only (under net metering)
    day_cost_pv_only = 0
    for i in range(len(net_load)):
        if net_load[i] > 0:  # Buying from grid
            day_cost_pv_only += net_load[i] * buy_prices[i]
        else:  # Selling to grid
            day_cost_pv_only += net_load[i] * sell_prices[i]  # Net_load is negative here
    
    # Optimize battery operation with price arbitrage
    # For net metering, the goal is to charge when prices are low and discharge when prices are high
    profit_value, p_c_value, p_d_value, X_value = Optimizer(battery_params, sell_prices, buy_prices, net_load)
    
    # Calculate new net load with battery
    net_load_with_battery = net_load + p_c_value - p_d_value
    
    # Calculate cost with PV and battery
    day_cost_battery = 0
    for i in range(len(net_load_with_battery)):
        if net_load_with_battery[i] > 0:  # Buying from grid
            day_cost_battery += net_load_with_battery[i] * buy_prices[i]
        else:  # Selling to grid
            day_cost_battery += net_load_with_battery[i] * sell_prices[i]
    
    # Add to yearly totals
    costs_pv_only[str(year)] += day_cost_pv_only
    costs_battery_only[str(year)] += day_cost_battery

# Calculate yearly benefits of having a battery
PV_benefit = {
    '2022': costs_nothing['2022'] - costs_pv_only['2022'],
    '2023': costs_nothing['2023'] - costs_pv_only['2023']
}
battery_benefits = {
    '2022': costs_pv_only['2022'] - costs_battery_only['2022'],
    '2023': costs_pv_only['2023'] - costs_battery_only['2023']
}
battery_only_benefits = {
    '2022': costs_nothing['2022'] - costs_battery_only['2022'],
    '2023': costs_nothing['2023'] - costs_battery_only['2023']
}

# Display results
print('--- 3.2 results ---')
print(f"Annual costs with PV only (DKK):")
print(f"2022: {costs_pv_only['2022']:.2f}")
print(f"2023: {costs_pv_only['2023']:.2f}")
print("\nAnnual benefits from PV system (DKK):")
print(f"2022: {PV_benefit['2022']:.2f}")
print(f"2023: {PV_benefit['2023']:.2f}")

print("\n\n--- 3.3 results ---")
print("Annual costs with PV and battery (DKK):")
print(f"2022: {costs_battery_only['2022']:.2f}")
print(f"2023: {costs_battery_only['2023']:.2f}")
print("\nAnnual benefits from battery (DKK):")
print(f"2022: {battery_benefits['2022']:.2f}")
print(f"2023: {battery_benefits['2023']:.2f}")
print("\nAnnual benefits from PV and battery (DKK):")
print(f"2022: {battery_only_benefits['2022']:.2f}")
print(f"2023: {battery_only_benefits['2023']:.2f}")

### 3.4
Computing the yearly benefit of only having a battery.

In [ ]:
# Load data
df_prices, df_pro = LoadData()
df_prices = PricesDK(df_prices)

# Battery parameters
battery_params = {
    'Pmax': 5,      # Power capacity in kW
    'Cmax': 10,     # Energy capacity in kWh
    'Cmin': 1,      # Minimum SOC (10%)
    'C_0': 5,       # Initial SOC (50%)
    'C_n': 5,       # Final SOC (50%)
    'n_c': 0.95,    # Charging efficiency
    'n_d': 0.95     # Discharging efficiency
}

# Initialize results dictionaries
costs_nothing = {'2022': annual_total_costs[2022], '2023': annual_total_costs[2023]}
costs_battery_only = {'2022': 0, '2023': 0}

# Process data by days
days = pd.to_datetime(df_prices['HourDK'].dt.date.unique())
for day in days:
    year = day.year
    day_date = day.date()
    
    # Filter data for the current day
    prices_day = df_prices[df_prices['HourDK'].dt.date == day_date]
    pro_day = df_pro[df_pro['HourDK'].dt.date == day_date]
    
    # Make sure we have both price and prosumer data for this day
    if len(prices_day) == 0 or len(pro_day) == 0:
        continue
        
    # Make sure the timestamps match
    merged = pd.merge(prices_day, pro_day, on='HourDK')
    if len(merged) == 0:
        continue
    
    # Extract data
    buy_prices = merged['Buy'].values
    sell_prices = merged['Sell'].values
    load = merged['Load'].values
    
    # Optimize battery operation with price arbitrage
    # For net metering, the goal is to charge when prices are low and discharge when prices are high
    profit_value, p_c_value, p_d_value, X_value = Optimizer(battery_params, sell_prices, buy_prices, load)
    
    # Calculate new net load with battery
    net_load_with_battery = load + p_c_value - p_d_value
    
    # Calculate cost with battery
    day_cost_battery = 0
    for i in range(len(net_load_with_battery)):
        if net_load_with_battery[i] > 0:  # Buying from grid
            day_cost_battery += net_load_with_battery[i] * buy_prices[i]
        else:  # Selling to grid
            day_cost_battery += net_load_with_battery[i] * sell_prices[i]
    
    costs_battery_only[str(year)] += day_cost_battery

# Calculate yearly benefits of having a battery
battery_only_benefits = {
    '2022': costs_nothing['2022'] - costs_battery_only['2022'],
    '2023': costs_nothing['2023'] - costs_battery_only['2023']
}

print("--- 3.4 results ---")
print("Annual costs of battery only (DKK):")
print(f"2022: {costs_battery_only['2022']:.2f}")
print(f"2023: {costs_battery_only['2023']:.2f}")

print("\nAnnual benefits from battery only (DKK):")
print(f"2022: {battery_only_benefits['2022']:.2f}")
print(f"2023: {battery_only_benefits['2023']:.2f}")